## Trying some classifiers... ##

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

### Using Pandas to import dataset ###

In [ ]:
all_data= pd.read_csv("../input/creditcard.csv")
labels = all_data["Class"].values
times = all_data["Time"].values
features = all_data.drop('Class', 1)
features = features.drop('Time', 1)
print(all_data.shape)
print(features.shape)
print(labels.shape)

### Using scikit-learn ###

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

Generate train and test sets (70/30):

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.3, random_state=0)

### Logistic regression ###

In [ ]:
lr = LogisticRegression(class_weight='balanced',random_state=0)
model_lr = lr.fit(features_train,labels_train)
y_predicted_lr = model_lr.predict(features_test)

In [ ]:
print('Confusion matrix (Logistic Regression)\n')
print(confusion_matrix(labels_test, y_predicted_lr))

In [ ]:
print('Classification report (Logistic Regression)\n')
print(classification_report(labels_test, y_predicted_lr, digits=5))

First method, high level in recall for fraud class (~91%), but a lot of false positives.

### Random Forest ###

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(class_weight='balanced',n_estimators=500,min_samples_leaf=20,random_state=0)
model_rf = rf.fit(features_train,labels_train)
y_predicted_rf = model_rf.predict(features_test)

In [ ]:
print('Confusion matrix (Random Forest)\n')
print(confusion_matrix(labels_test, y_predicted_rf))

In [ ]:
print('Classification report (Random Forest)\n')
print(classification_report(labels_test, y_predicted_rf, digits=5))

Using Random Forest, we can obtain less recall on fraud class (~82%), but the number of false positives has dropped dramatically.

### Neural Network (Multi-Layer Perceptron) ###

A two hidden-layers neural network, with perceptron output.

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(solver='lbfgs', activation='tanh', alpha=1, hidden_layer_sizes=(58,58), learning_rate='adaptive', random_state=0)
model_mlp = mlp.fit(features_train,labels_train)
y_predicted_mlp = model_mlp.predict(features_test)

In [ ]:
print('Confusion matrix (Multi-layer Perceptron)\n')
print(confusion_matrix(labels_test, y_predicted_mlp))

In [ ]:
print('Classification report (Multi-layer Perceptron)\n')
print(classification_report(labels_test, y_predicted_mlp, digits=5))

Using a Multi-layer perceptron, with `tanh` activators, we can obtain similar results than using a Random Forest (recall ~80%, low false positives). But watch out! Training time is high!

### Decision Tree ###

In [ ]:
from sklearn import tree
tree_clf = tree.DecisionTreeClassifier(random_state=0)
tree_model = tree_clf.fit(features_train,labels_train)
y_predicted_tree = tree_model.predict(features_test)

In [ ]:
print('Confusion matrix (Decision Tree)\n')
print(confusion_matrix(labels_test, y_predicted_tree))

In [ ]:
print('Classification report (Decision Tree)\n')
print(classification_report(labels_test, y_predicted_tree, digits=5))

Decision tree works a little worse than the rest. Recall ~76%.

## Subsample of majority class ##

In order to deal with a highly skewed dataset, we can randomly subsample the majority class (not fraud, class=0). We are going to keep only 10000 samples of class 0.

In [ ]:
features_class0 = features[labels == 0]
features_class1 = features[labels == 1]
print('FEATURES')
print('Class 0: ', features_class0.shape, 'Class 1: ', features_class1.shape)
labels_class0 = labels[(labels == 0)]
labels_class1 = labels[(labels == 1)]
print('LABELS')
print('Class 0: ', labels_class0.shape, 'Class 1: ', labels_class1.shape)

In [ ]:
features_class0_sub = features_class0.sample(n=10000, random_state=0)
labels_class0_sub = labels_class0[:10000]
print('Class 0 :', features_class0_sub.shape[0], 'Class 1: ', features_class1.shape[0])

features_sub = np.concatenate((features_class0_sub,features_class1))
labels_sub = np.concatenate((labels_class0_sub,labels_class1))
features_sub_train, features_sub_test, labels_sub_train, labels_sub_test = train_test_split(
    features_sub, labels_sub, test_size=0.3, random_state=0)

### Random Forest (with subsample) ###

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=500,min_samples_leaf=20,random_state=0)
model_rf_sub = rf.fit(features_sub_train,labels_sub_train)
y_predicted_rf_sub = model_rf_sub.predict(features_sub_test)

In [ ]:
print('Confusion matrix (Random Forest, with subsample)\n')
print(confusion_matrix(labels_sub_test, y_predicted_rf_sub))

In [ ]:
print('Classification report (Random Forest, with subsample)\n')
print(classification_report(labels_sub_test, y_predicted_rf_sub, digits=5))

Recall is a little higher than before (~83,7%), and we don't have false positives!

### Neural Network (Multi-Layer Perceptron, with subsample) ###

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(solver='lbfgs', activation='tanh', alpha=1, hidden_layer_sizes=(58, 58), learning_rate='adaptive', random_state=0)
model_mlp_sub = mlp.fit(features_sub_train,labels_sub_train)
y_predicted_mlp_sub = model_mlp_sub.predict(features_sub_test)

In [ ]:
print('Confusion matrix (Multi-layer Perceptron, with subsample)\n')
print(confusion_matrix(labels_sub_test, y_predicted_mlp_sub))

In [ ]:
print('Classification report (Multi-layer Perceptron, with subsample)\n')
print(classification_report(labels_sub_test, y_predicted_mlp_sub, digits=5))

Finally, last example give us the higher recall for fraud detection (~86,5%). Very low false positive and also low false negatives.